In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats

import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sb # visualization

from sklearn.model_selection import train_test_split # data split
from sklearn.linear_model import LinearRegression # OLS algorithm

from sklearn.metrics import explained_variance_score as evs # evaluation metric
from sklearn.metrics import r2_score as r2 # evaluation metric
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn import svm
from sklearn.svm import SVC

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


sb.set_style('whitegrid') # plot style

In [18]:
#LINHA DE SELEÇÃO DO INPUT
df = pd.read_csv('cade_clinicas_2.csv', low_memory=False)

df.columns


Index(['id', 'id_unique', 'numero_sei', 'ano_documento', 'assinaturas',
       'descricao_tipo_acesso', 'descricao_tipo_documento',
       'descricao_tipo_processo', 'numero_processo', 'id_unidade',
       'sigla_unidade', 'descricao_unidade', 'data_ordem', 'data_documento',
       'setor_economico', 'setor_economico_classe', 'setor_economico_divisao',
       'setor_economico_grupo', 'setor_economico_secao',
       'setor_economico_sub_classe', 'partes_processos_like',
       'partes_processos', 'decisao_sg', 'dados_processo',
       'ementa_dispositivo', 'ementa_verbetacao', 'id_colecao',
       'link_documento', 'mercado_relevante', 'documentos_relacionados',
       'qtd_acesso_processo', 'descricao_tipo_protocolo',
       'descricao_especificacao', 'data_processo', 'colecao', '_version_',
       'palavras', 'relator', 'score', 'nucluster', 'sumario',
       'representante_legal', 'descricao_assunto', 'quorum', 'tags',
       'entidade', 'numero_acordao', 'interessados', 'numero_ata'

Filtro por Voto

In [88]:
tipos_validos = [
    "Voto",
    "Voto Processo Administrativo",
    "Voto Embargos de Declaração"
]

df_votos_filtrados = df.loc[
    (df["descricao_tipo_documento"].isin(tipos_validos)) &
    (df["descricao_tipo_processo"] == "Processo Administrativo")
]

counts = df_votos_filtrados["decisao_tribunal"].value_counts()
total = counts.sum()
print(total)

print(df_votos_filtrados['descricao_titulo_documento'].head(20))


908
0     NaN
40    NaN
43    NaN
85    NaN
86    NaN
87    NaN
88    NaN
97    NaN
197   NaN
220   NaN
261   NaN
272   NaN
273   NaN
281   NaN
287   NaN
541   NaN
542   NaN
550   NaN
554   NaN
574   NaN
Name: descricao_titulo_documento, dtype: float64


Limpando colunas

In [ ]:
colunas_para_remover = [
    'id_colecao','id_unique', '_version_', 'clicks', 'soundex', 'vltfidf',
    'qtd_acesso',  'documentos_citados', 'tags', 'numero_acordao', 'numero_ata',
    'nucluster', 'score', 'sumario', 'colecao', 'palavras',  'qtd_acesso_processo'
]
colunas_existentes = [col for col in colunas_para_remover if col in df.columns]
df_votos_filtrados = df_votos_filtrados.drop(columns=colunas_existentes)

#pd.set_option('display.max_colwidth', 10000)



0    SEI/CADE - 1390607 - Voto Processo Administrativo\r\n\r\n\r\nProcesso Administrativo nº 08700.004558/2019-05 (Apartado de Acesso Restrito nº 08700.004559/2019-41)\r\n\r\nRepresentante: CADE ex officio\r\n\r\nRepresentados(as): Bernd Brünig, Faustino Luigi Minchella e José Angel Viani Barroyeta \r\n\r\nRelator(a): Conselheiro Carlos Jacques Vieira Gomes \r\nVOTO DO RELATOR - CONSELHEIRO-RELATOR CARLOS JACQUES VIEIRA GOMES\r\n\r\nVERSÃO DE ACESSO PÚBLICO\r\n\r\n\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tEmenta:\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tProcesso Administrativo. CARTEL de peças automotivas. mercado independente de peças de reposição (“IAM”). mercado peças originais (“OEM”). PESSOAS FÍSICAS. DESMEMBRAMENTO. PARECER DA SUPERINTENDÊNCIA-GERAL PELA  CONDENAÇÃO parcial.  PARECER DA PROCURADORIA FEDERAL ESPECIALIZADA PELA CONDENAÇÃO parcial. PARECER DO MINISTÉRIO PÚBLICO FEDERAL PELA CONDENAÇÃO  PARCIAL. revelia. necessidade de corroboração de declarações de compromissários e lenientes. VOTO PELA CO

In [96]:
colunas_manter = [ 'id', 'ano_documento', 'assinaturas', 'descricao_tipo_documento',
                  'decisao_tribunal', 'decisao_sg' , 'setor_economico', 'id_unidade',
                  'data_ordem', 'data_processo', 'data_documento', 'mercado_relevante',
                'descricao_especificacao', 'conteudo'      
]

#print(df_votos_filtrados[colunas_manter].info())

df_curto = df_votos_filtrados[colunas_manter]

pd.set_option('display.max_colwidth', 100)
display(df_curto.head(10))
print(df_curto.shape)

,id,ano_documento,assinaturas,descricao_tipo_documento,decisao_tribunal,decisao_sg,setor_economico,id_unidade,data_ordem,data_processo,data_documento,mercado_relevante,descricao_especificacao,conteudo
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,2024,['Outras'],Voto Processo Administrativo,[''],['Condenação'],['Fabricação de Peças e Acessórios para o Sistema Motor de Veículos Automotores'],110000967.0,20240523,2019-09-13T00:00:00Z,2024-05-23T00:00:00Z,"['pistões de motor, bronzinas, camisas, pinos, bielas, porta anéis, anéis e juntas de vedação, e...","- Desmembrado Bernd Brünig, Faustino Luigi Minchella e José Angel Viani Barroyeta",SEI/CADE - 1390607 - Voto Processo Administrativo\r\n\r\n\r\nProcesso Administrativo nº 08700.00...
40,b41e8f1f-449e-4d28-a574-18f365d32b6f,2024,['Gustavo Augusto Freitas de Lima'],Voto Processo Administrativo,['Condenação'],['Condenação'],['Gestão e Administração da Propriedade Imobiliária'],110000969.0,20240509,2015-05-21T00:00:00Z,2024-05-09T00:00:00Z,['Mercado imobiliário'],Influencia de conduta uniforme - COFECI,SEI/CADE - 1384594 - Voto Processo Administrativo\r\n\r\n\r\nPedido de Reapreciação no Processo ...
43,adec625d-4f5e-4514-b938-a295fdc1e8a4,2024,['Outras'],Voto,['Arquivamento'],['Arquivamento'],"['Comércio Atacadista de Combustíveis Sólidos, Líquidos e Gasosos, Exceto Gás Natural e Glp']",110001021.0,20240509,2022-08-08T00:00:00Z,2024-05-09T00:00:00Z,['Postos de gasolina'],Ministério Público do Estado de Minas Gerais,SEI/CADE - 1384685 - Voto\r\n\r\n\r\nProcesso nº 08700.005915/2022-40\r\n\r\nRepresentante: Cade...
85,c8499708-5778-45d9-8914-9ab97b77c1fd,2024,['Alexandre Cordeiro Macedo'],Voto Processo Administrativo,['Condenação'],['Condenação'],['Construção de Edifícios# Construção de Rodovias e Ferrovias'],110000955.0,20240423,2016-11-22T00:00:00Z,2024-04-23T00:00:00Z,['obras de engenharia civil# Construção de infraestrutura de rodovias e ferrovias'],PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377990 - Voto Processo Administrativo\r\n\r\n\r\nProcesso Administrativo nº 08700.00...
86,29758e3d-7047-4c5a-8ac6-b6815c67ec3d,2024,['Gustavo Augusto Freitas de Lima'],Voto Processo Administrativo,['Condenação'],['Condenação'],['Construção de Edifícios# Construção de Rodovias e Ferrovias'],110000969.0,20240423,2016-11-22T00:00:00Z,2024-04-23T00:00:00Z,['obras de engenharia civil# Construção de infraestrutura de rodovias e ferrovias'],PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377023 - Voto Processo Administrativo\r\n\r\n\r\nProcesso Administrativo nº 08700.00...
87,15187e9b-d1e4-4c9d-97e2-721755174c1c,2024,['Outras'],Voto Processo Administrativo,['Condenação'],['Condenação'],['Construção de Edifícios# Construção de Rodovias e Ferrovias'],110000972.0,20240423,2016-11-22T00:00:00Z,2024-04-23T00:00:00Z,['obras de engenharia civil# Construção de infraestrutura de rodovias e ferrovias'],PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377847 - Voto Processo Administrativo\r\n\r\n\r\n \r\n\r\nProcesso Administrativo nº...
88,2aabf3bb-af87-47ed-b09b-0726d04cdf5f,2024,['Outras'],Voto Processo Administrativo,['Condenação'],['Condenação'],['Construção de Edifícios# Construção de Rodovias e Ferrovias'],110000967.0,20240423,2016-11-22T00:00:00Z,2024-04-23T00:00:00Z,['obras de engenharia civil# Construção de infraestrutura de rodovias e ferrovias'],PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377767 - Voto Processo Administrativo\r\n\r\n\r\nProcesso Administrativo nº 08700.00...
97,0d278d0f-5ab8-44cc-9ec6-cde5c399d9f9,2024,['Gustavo Augusto Freitas de Lima'],Voto Processo Administrativo,['Arquivamento'],['Arquivamento'],['Comércio de Peças e Acessórios para Veículos Automotores'],110000969.0,20240422,2022-05-17T00:00:00Z,2024-04-22T00:00:00Z,['Revenda/distribuição de pneus'],Autos públicos,SEI/CADE - 1375782 - Voto Processo Administrativo\r\n\r\n\r\nProcesso Administrativo nº 08700.00...
197,84fe3a4b-a1b4-4536-957e-2aa5c372527f,2024,['Gustavo Augusto Freitas de Lima'],Voto Processo Administrativo,[''],

(908, 14)
